# Geospatial Data Analysis Lab: Steel Plants Dataset


**(15/10/2025) Learning Objectives:**
- Perform exploratory data analysis (EDA) on geospatial datasets
- Visualize geospatial data using interactive maps with Plotly
- Merge environmental data with asset locations
- Aggregate data at the company level
- Integrate geospatial visualizations into a Streamlit dashboard

---


## Part 1: Setup and Data Loading

Import the necessary libraries and load the steel plants dataset.


### Imports

In [1]:
import os
from pathlib import Path

import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go

from functools import reduce

import requests
from tqdm.notebook import tqdm

import tarfile

from pyspark.sql import SparkSession

import geopandas as gpd
from shapely.geometry import Point

### HyperParams

In [2]:
XLSX_DATA_PATH = Path("data/dataSept2025.xlsx")
ON_COLAB = True # Put false if executed in another environement than Google Colab

### Functions

In [3]:
def get_dataframe(
    file_path : str,
    sheet_names : list[str],
    key_col:str):

  read_df = lambda s : pd.read_excel(file_path, sheet_name=s)
  dfs = [read_df(s) for s in sheet_names]

  for df in dfs:
    df.set_index(key_col, inplace=True)

  return reduce(lambda left, right: pd.merge(left, right, on=key_col), dfs)

### Run

In [4]:
merged_df = get_dataframe(XLSX_DATA_PATH, ["Plant data", "Plant capacities and status"], "Plant ID")
merged_df.head()


,Plant name (English)_x,Plant name (other language)_x,Other plant names (English),Other plant names (other language),Owner,Owner (other language),Owner GEM ID,Owner PermID,SOE Status,Parent,...,Start date_y,Nominal crude steel capacity (ttpa),Nominal BOF steel capacity (ttpa),Nominal EAF steel capacity (ttpa),Nominal OHF steel capacity (ttpa),Other/unspecified steel capacity (ttpa),Nominal iron capacity (ttpa),Nominal BF capacity (ttpa),Nominal DRI capacity (ttpa),Other/unspecified iron capacity (ttpa)
Plant ID,,,,,,,,,,,,,,,,,,,,,
P100000120004,Kurum International Elbasan steel plant,Kurum Kombinati metalurgjik,NaN,NaN,Kurum International ShA,NaN,E100000130992,5037939021,NaN,Kurum International ShA [100.0%],...,1998,700,NaN,700,NaN,NaN,NaN,NaN,NaN,NaN
P100000120439,Algerian Qatari Steel Jijel plant,الجزائرية القطرية للصلب,AQS,NaN,Algerian Qatari Steel,NaN,E100001000957,5076384326,Partial,Industries Qatar QPSC [49.0%]; unknown [5.0%];...,...,2017,2200,NaN,2200,NaN,NaN,2500,NaN,2500,NaN
P100000120439,Algerian Qatari Steel Jijel plant,الجزائرية القطرية للصلب,AQS,NaN,Algerian Qatari Steel,NaN,E100001000957,5076384326,Partial,Industries Qatar QPSC [49.0%]; unknown [5.0%];...,...,2026,1800,NaN,1800,NaN,NaN,2500,NaN,2500,NaN
P100000120442,ETRHB Annaba steel plant,NaN,NaN,NaN,ETRHB Industrie SpA,NaN,E100001010275,5074513855,NaN,ETRHB Industrie SpA [100.0%],...,unknown,1150,NaN,1150,NaN,NaN,NaN,NaN,NaN,NaN
P100000121198,Ozmert Algeria steel plant,NaN,NaN,NaN,Ozmert Algeria SARL,NaN,E100001012196,unknown,NaN,Ozmert Algeria SARL [100.0%],...,unknown,800,NaN,800,NaN,NaN,500,NaN,500,NaN


In [5]:
merged_df.columns

Index(['Plant name (English)_x', 'Plant name (other language)_x',
       'Other plant names (English)', 'Other plant names (other language)',
       'Owner', 'Owner (other language)', 'Owner GEM ID', 'Owner PermID',
       'SOE Status', 'Parent', 'Parent GEM ID', 'Parent PermID',
       'Location address', 'Municipality', 'Subnational unit (province/state)',
       'Country/Area_x', 'Region', 'Other language location address',
       'Coordinates', 'Coordinate accuracy', 'GEM wiki page',
       'Plant age (years)', 'Announced date', 'Construction date',
       'Start date_x', 'Pre-retirement announcement date', 'Idled date',
       'Retired date', 'Ferronickel capacity (ttpa)',
       'Sinter plant capacity (ttpa)', 'Coking plant capacity (ttpa)',
       'Pelletizing plant capacity (ttpa)', 'Category steel product',
       'Steel products', 'Steel sector end users', 'Workforce size',
       'ISO 14001', 'ISO 50001', 'ResponsibleSteel Certification',
       'Main production equipment', 

In [6]:
merged_df["Status"].unique()

array(['mothballed', 'operating', 'announced', 'cancelled', 'retired',
       'operating pre-retirement', 'construction',
       'mothballed pre-retirement'], dtype=object)

In [7]:
operating_df = merged_df[merged_df["Status"].isin(["operating", "operating pre-retirement"])]

operating_df.head()

,Plant name (English)_x,Plant name (other language)_x,Other plant names (English),Other plant names (other language),Owner,Owner (other language),Owner GEM ID,Owner PermID,SOE Status,Parent,...,Start date_y,Nominal crude steel capacity (ttpa),Nominal BOF steel capacity (ttpa),Nominal EAF steel capacity (ttpa),Nominal OHF steel capacity (ttpa),Other/unspecified steel capacity (ttpa),Nominal iron capacity (ttpa),Nominal BF capacity (ttpa),Nominal DRI capacity (ttpa),Other/unspecified iron capacity (ttpa)
Plant ID,,,,,,,,,,,,,,,,,,,,,
P100000120439,Algerian Qatari Steel Jijel plant,الجزائرية القطرية للصلب,AQS,NaN,Algerian Qatari Steel,NaN,E100001000957,5076384326,Partial,Industries Qatar QPSC [49.0%]; unknown [5.0%];...,...,2017,2200,NaN,2200,NaN,NaN,2500,NaN,2500,NaN
P100000121198,Ozmert Algeria steel plant,NaN,NaN,NaN,Ozmert Algeria SARL,NaN,E100001012196,unknown,NaN,Ozmert Algeria SARL [100.0%],...,unknown,800,NaN,800,NaN,NaN,500,NaN,500,NaN
P100000120440,Sider El Hadjar Annaba steel plant,مركب الحجار للحديد والصلب,"ArcelorMittal Annaba (predecessor), El Hadjar ...",NaN,Groupe Industriel Sider SpA,NaN,E100001000960,5000941519,Full,Groupe Industriel Sider SpA [100.0%],...,1969,2150,350.0,1800,NaN,NaN,1500,1500,NaN,NaN
P100000120441,Tosyali Algerie Oran steel plant,شركة توسيالي الجزائرية التركية للحديد والصلب,NaN,Tosyali Algérie,Tosyali Ironsteel Industry Algerie SpA,NaN,E100000131071,5074196906,NaN,Tosyalı Holding AŞ,...,2013,6200,NaN,6200,NaN,NaN,5000,NaN,5000,NaN
P100000120005,Aceria Angola Bengo steel plant,NaN,ADA Steel,NaN,Ada - Aceria De Angola SA,NaN,E100000131097,unknown,NaN,Sa Machado Lda,...,2015,500,NaN,NaN,NaN,500,NaN,NaN,NaN,NaN


---
## Part 2: Exploratory Data Analysis (15 minutes)

Answer the following questions through your analysis:


### Question 1: Data Overview
**Task:** Display basic information about the dataset.
- How many steel plants are in the dataset?
- What are the column names and data types?
- Are there any missing values?


### Functions

In [8]:
def get_missingVals_df(
    df):

  missing = df.isna().sum().sort_values(ascending=False)
  missing_df = missing[missing > 0] if missing.sum() > 0 else pd.Series([], dtype=int)
  return missing_df

def get_infos_df(df):
  return pd.DataFrame({"column": df.columns,"dtype": df.dtypes.astype(str)})

### Run

In [9]:
# Display dataset shape
merged_df.shape

(1744, 57)

In [10]:
# Display column information and data types
info_df = get_infos_df(merged_df)
display(info_df)

,column,dtype
Plant name (English)_x,Plant name (English)_x,object
Plant name (other language)_x,Plant name (other language)_x,object
Other plant names (English),Other plant names (English),object
Other plant names (other language),Other plant names (other language),object
Owner,Owner,object
Owner (other language),Owner (other language),object
Owner GEM ID,Owner GEM ID,object
Owner PermID,Owner PermID,object
SOE Status,SOE Status,object
Parent,Parent,object


In [11]:
# Check for missing values
missing_df = get_missingVals_df(merged_df)
display(missing_df)

,0
Nominal OHF steel capacity (ttpa),1739
Other/unspecified iron capacity (ttpa),1712
Other/unspecified steel capacity (ttpa),1625
Nominal DRI capacity (ttpa),1474
SOE Status,1445
Other plant names (other language),1268
Nominal BOF steel capacity (ttpa),1128
Other language location address,1027
Nominal BF capacity (ttpa),1013
Owner (other language),884


### Question 2: Statistical Summary
**Task:** Generate descriptive statistics for numerical columns.
- What is the average plant capacity?
- What is the range of latitudes and longitudes?
- What is the distribution of plant ages?


#### Functions

In [12]:
def convert_to_numeric(df, cols):
    for col in cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

def split_coordinatesCol(df, col = "Coordinates"):

    if col not in df.columns:
        print(f"Column '{col}' not found in the dataframe.")
        return df

    # Assuming 'Coordinates' is a string like 'lat, lon'
    coords_split = df[col].str.split(',', expand=True)
    df['latitude'] = pd.to_numeric(coords_split[0], errors='coerce')
    df['longitude'] = pd.to_numeric(coords_split[1], errors='coerce')
    return df

def groupBy_sum(df, key_col, col):
  if not key_col in df.columns:
    raise ValueError(f"Key column '{key_col}' not found in the dataframe.")

  if not col in df.columns:
    raise ValueError(f"Column '{col}' not found in the dataframe.")

  return (df.groupby(df[key_col].fillna("Unknown"))[[col]]
        .sum()
        .sort_values(col, ascending=False)
        .head(20)
    )

#### Run

In [13]:
# Convert relevant columns to numeric, coercing errors
numeric_cols = [
    "Nominal crude steel capacity (ttpa)",
    "Nominal BOF steel capacity (ttpa)",
    "Nominal EAF steel capacity (ttpa)",
    "Nominal OHF steel capacity (ttpa)",
    "Other/unspecified steel capacity (ttpa)",
    "Nominal iron capacity (ttpa)",
    "Nominal BF capacity (ttpa)",
    "Nominal DRI capacity (ttpa)",
    "Other/unspecified iron capacity (ttpa)",
    "Plant age (years)",
]

operating_df = convert_to_numeric(operating_df, numeric_cols)
operating_df = split_coordinatesCol(operating_df)

# Display results
print("Descriptive statistics for numerical columns:")
display(operating_df.describe())
print("\nRange of Latitudes and Longitudes:")
display(operating_df[['latitude', 'longitude']].agg(['min', 'max']))


Descriptive statistics for numerical columns:


/tmp/ipython-input-3464368244.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pd.to_numeric(df[col], errors='coerce')
/tmp/ipython-input-3464368244.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['latitude'] = pd.to_numeric(coords_split[0], errors='coerce')
/tmp/ipython-input-3464368244.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

,Plant age (years),Nominal crude steel capacity (ttpa),Nominal BOF steel capacity (ttpa),Nominal EAF steel capacity (ttpa),Nominal OHF steel capacity (ttpa),Other/unspecified steel capacity (ttpa),Nominal iron capacity (ttpa),Nominal BF capacity (ttpa),Nominal DRI capacity (ttpa),Other/unspecified iron capacity (ttpa),latitude,longitude
count,977.000000,915.000000,370.000000,556.000000,3.000000,50.000000,515.000000,422.000000,122.000000,5.000000,993.000000,993.000000
mean,40.772907,2403.566689,3937.837838,1263.375899,1726.666667,692.930400,3197.027184,3508.443128,1283.434426,1865.400000,31.148695,63.306755
std,36.244503,2783.251227,3350.117739,993.840682,1107.534800,798.720471,3215.476247,3279.897712,1398.962408,1457.201359,16.391615,68.835872
min,0.000000,13.000000,334.000000,13.000000,750.000000,48.000000,9.000000,33.000000,1.000000,427.000000,-37.831379,-123.163599
25%,17.780000,800.000000,1712.500000,679.500000,1125.000000,289.750000,1100.000000,1200.000000,376.000000,1200.000000,24.465884,19.646010
50%,27.000000,1350.000000,3000.000000,1000.000000,1500.000000,500.000000,2130.000000,2470.000000,860.000000,1700.000000,34.503883,91.715778
75%,56.000000,2968.000000,4852.500000,1411.250000,2215.000000,760.000000,4305.000000,4620.000000,1690.000000,1700.000000,40.508993,117.457779
max,286.000000,22999.000000,22999.000000,10000.000000,2930.000000,5000.000000,24750.000000,24750.000000,7830.000000,4300.000000,66.311519,174.728098



Range of Latitudes and Longitudes:


,latitude,longitude
min,-37.831379,-123.163599
max,66.311519,174.728098


In [14]:
# Group by company and sum capacity
company_capacity = groupBy_sum(operating_df, "Owner", "Nominal crude steel capacity (ttpa)")
print("\nTop 20 Companies by Total Crude Steel Capacity:")
display(company_capacity)


Top 20 Companies by Total Crude Steel Capacity:


,Nominal crude steel capacity (ttpa)
Owner,
Nippon Steel Corp,44423.0
POSCO Holdings Inc,40700.0
Angang Steel Co Ltd,30250.0
JFE Steel Corp,28468.0
JSW Steel Ltd,28359.0
Cleveland-Cliffs Inc,26377.0
Hyundai Steel Co,24297.0
Steel Authority of India Ltd,21050.0
Baoshan Iron & Steel Co Ltd,19800.0


### Question 3: Geographic Distribution
**Task:** Analyze the geographic distribution of steel plants.
- Which countries/regions have the most steel plants?
- What is the distribution of plants by company?


#### Hyperparams

In [15]:
NB_BEST = 10

#### Functions

In [16]:
# Count plants by country/region
country_counts = operating_df["Country/Area_x"].value_counts()

print(f"\nTop {NB_BEST} Countries by Number of Operating Steel Plants:")
display(country_counts.head(NB_BEST))


Top 10 Countries by Number of Operating Steel Plants:


,count
Country/Area_x,
China,371
India,78
United States,71
Japan,43
Iran,31
Russia,29
Türkiye,25
Italy,22
Brazil,22


In [17]:
# Count plants by company
company_plant_counts = operating_df["Owner"].value_counts()

print(f"\nTop {NB_BEST} Companies by Number of Operating Steel Plants:")
display(company_plant_counts.head(NB_BEST))


Top 10 Companies by Number of Operating Steel Plants:


,count
Owner,
Cleveland-Cliffs Inc,11
Nucor Corp,11
Nippon Steel Corp,11
Steel Authority of India Ltd,8
Gerdau Ameristeel Corp,7
Commercial Metals Co,7
Steel Dynamics Inc,6
ArcelorMittal Brasil SA,6
ArcelorMittal SA,6


### Question 4: Capacity Analysis
**Task:** Analyze the capacity distribution.
- What is the total global steel production capacity?
- Which companies have the highest total capacity?
- How does capacity vary by region?


#### fucntions

#### Run

In [18]:
# Calculate total global crude steel capacity
total_global_capacity = operating_df["Nominal crude steel capacity (ttpa)"].sum()
print(f"Total global crude steel capacity from operating plants: {total_global_capacity:,.0f} ttpa")

Total global crude steel capacity from operating plants: 2,199,264 ttpa


In [19]:
# Which companies have the highest total capacity?
company_total_capacity = operating_df.groupby("Owner")["Nominal crude steel capacity (ttpa)"].sum().sort_values(ascending=False)

print(f"\nTop {NB_BEST} Companies by Total Crude Steel Capacity:")
display(company_total_capacity.head(NB_BEST))


Top 10 Companies by Total Crude Steel Capacity:


,Nominal crude steel capacity (ttpa)
Owner,
Nippon Steel Corp,44423.0
POSCO Holdings Inc,40700.0
Angang Steel Co Ltd,30250.0
JFE Steel Corp,28468.0
JSW Steel Ltd,28359.0
Cleveland-Cliffs Inc,26377.0
Hyundai Steel Co,24297.0
Steel Authority of India Ltd,21050.0
Baoshan Iron & Steel Co Ltd,19800.0


---
## Part 3: Geospatial Visualization with Plotly (15 minutes)

Create interactive maps to visualize the steel plants' locations and characteristics.


### Exercise 1: Basic Scatter Map
**Task:** Create a scatter map showing all steel plant locations.
- Use latitude and longitude for positioning
- Color points by country or region
- Add hover information showing plant name, company, and capacity


#### Functions

In [20]:
def getFig_basic_scatter(df, title, color, hover_data, **kwargs):
  if ('latitude' not in df.columns or 'longitude' not in df.columns) and 'lat' not in kwargs and 'lon' not in kwargs:
    raise ValueError("Latitude and Longitude columns not found in the dataframe.")

  if "size" in kwargs:
    df[kwargs.get("size")] = df[kwargs.get("size")].fillna(0)

  fig = px.scatter_mapbox(
        df,
        lat=kwargs.get("lat", "latitude"),
        lon=kwargs.get("lon", "longitude"),
        color=color,
        hover_name="Plant name (English)_x",
        hover_data=hover_data,
        title=title,
        zoom=1,
        height=600,
        size = kwargs.get("size", None))
  fig.update_layout(mapbox_style="open-street-map")
  fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
  return fig

#### Run

In [21]:
# Create a scatter_geo or scatter_mapbox plot
# Hint: Use plotly.express.scatter_geo() or scatter_mapbox()

# Create a scatter_mapbox plot to visualize plant locations
fig = getFig_basic_scatter(operating_df,
                           title="Geographic Distribution of Operating Steel Plants",
                           color="Country/Area_x",
                           hover_data={"Owner": True,"Nominal crude steel capacity (ttpa)": True,"latitude": False,"longitude": False,"Country/Area_x": False}
                           )
fig.show()


### Exercise 2: Sized Markers by Capacity
**Task:** Create a map where marker size represents plant capacity.
- Larger markers for higher capacity plants
- Color by company
- Include interactive hover details


In [22]:
missing_capacity = operating_df['Nominal crude steel capacity (ttpa)'].isna().sum()
print(f"Number of missing values in 'Nominal crude steel capacity (ttpa)': {missing_capacity}")

Number of missing values in 'Nominal crude steel capacity (ttpa)': 78


In [23]:
fig = getFig_basic_scatter(operating_df,
                           title="Operating Steel Plants by Capacity and Owner",
                           color="Owner",
                           hover_data={"Owner": True,"Nominal crude steel capacity (ttpa)": True,"latitude": False,"longitude": False},
                           size="Nominal crude steel capacity (ttpa)"
                           )
fig.show()

/tmp/ipython-input-1134768948.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Exercise 3: Density Heatmap
**Task:** Create a density map showing concentration of steel plants.
- Use Plotly's density_mapbox to show clustering
- Identify regions with high plant density


In [24]:
# Create density heatmap
# Hint: Use plotly.express.density_mapbox()
fig = px.density_mapbox(
        operating_df,
        lat="latitude",
        lon="longitude",
        radius=10,  # Adjust radius as needed to control the size of the density circles
        center=dict(lat=0, lon=0),
        zoom=0,
        mapbox_style="open-street-map",
        title="Density Heatmap of Operating Steel Plants")
fig.show()


---
## Part 4: Merging Environmental Data with Assets

Integrate environmental data (e.g., air quality, emissions, proximity to water sources) with steel plant locations.


### Exercise 1: Load Environmental Data
**Task:** Load the environmental dataset and inspect it.

- [Litpop database](https://www.research-collection.ethz.ch/entities/researchdata/12dcfc4f-9d03-463a-8d6b-76c0dc73cdc8)

- Expected columns: location_id, latitude, longitude, population density, activity etc.


#### Hyperparams

In [25]:
DATA_SOURCE = "original"

#### Functions

In [26]:
def download_file(url, filename):
    # Stream download so we don't load it all into memory
    print("Downloading file from URL: ", url)
    response = requests.get(url, stream=True)
    response.raise_for_status()  # Raise error if bad status

    print("Response status: ", response.status_code)

    # Get total file size from headers if available
    total_size_in_bytes = int(response.headers.get('content-length', 0))
    block_size = 1024  # 1 KB

    print("Total size in bytes: ", total_size_in_bytes)

    # Set up the progress bar
    #

    mb_dwn : int = 0
    print("Downloading file to: ", filename)
    progress_bar = tqdm(total=1_000, unit='Mb', unit_scale=True, desc="Downloading 1GB (0)")

    #if not "data" folder, create it
    if not os.path.exists("data"):
        os.makedirs("data")

    with open("data/" + filename, 'wb') as file:
        for data in response.iter_content(block_size):
            # progress_bar.update(len(data))


            if not mb_dwn % 1000:
                progress_bar.update(1)

            if not mb_dwn % 1_000_000:
              progress_bar.close()
              progress_bar = tqdm(total=1_000, unit='Mb', unit_scale=True, desc=f"Downloading 1GB ({(mb_dwn // 1_000_000)+1})")

            mb_dwn += 1
            file.write(data)

    try:
      progress_bar.close()
    except:
      pass

def extract_tar(tar_filename, extract_path):
  try:
    with tarfile.open(tar_filename, "r") as tar:
        tar.extractall(path=extract_path)
    print(f"Successfully extracted {tar_filename} to {extract_path}")

  except tarfile.ReadError:
      print(f"Error: Could not read the tar file {tar_filename}. It might be corrupted or not a valid tar file.")
  except FileNotFoundError:
      print(f"Error: The file {tar_filename} was not found.")
  except Exception as e:
      print(f"An unexpected error occurred during extraction: {e}")

  return extract_path

def check_cols_in(df,
                  required_cols):
  try:
      # Check for required columns

      missing_cols = [col for col in required_cols if col not in df.columns]

      if not missing_cols:
          print("\nAll required columns are present.")
      else:
          print(f"\nMissing required columns: {missing_cols}")

  except FileNotFoundError:
      print(f"Error: The file {file_path} was not found. Please ensure the extraction was successful and the file exists.")
  except Exception as e:
      print(f"An error occurred while reading the CSV file: {e}")

  return df

def csv_to_spark(spark, csv_files : list):

  # Read each CSV file and append to a list of DataFrames
  dataframes = []
  if csv_files:
      print(f"Reading {len(csv_files)} CSV files...")
      for file in tqdm(csv_files, desc="Loading CSV files"):
          try:
              # Read each CSV file individually
              df = spark.read.csv(file, header=True, inferSchema=True)
              dataframes.append(df)
          except Exception as e:
              print(f"Error reading file {file}: {e}")

      # Union all DataFrames into a single Spark DataFrame
      if dataframes:
          env_spark_df = dataframes[0]
          for df in dataframes[1:]:
              env_spark_df = env_spark_df.unionAll(df)
      else:
          print("No DataFrames were successfully loaded.")
  else:
      raise Exception("No CSV files found in the specified directory.")

  return env_spark_df

#### Run

In [27]:
litPop_datasources = {
    "original" : "https://www.research-collection.ethz.ch/bitstreams/faadb23b-0d1d-4e12-893b-5251419d9822/download",
    "normalized" : "https://www.research-collection.ethz.ch/bitstreams/124516f7-53ea-4a86-ad01-7cf18f82a4fd/download"
}
filename = f"LitPop_v1_2_{DATA_SOURCE[:4].lower()}"

if DATA_SOURCE not in litPop_datasources:
    raise ValueError(f"Invalid DATA_SOURCE: {DATA_SOURCE}")

url = litPop_datasources[DATA_SOURCE]
# Replace with the full URL to the file
download_file(url, filename + '.tar')

Response status:  200
Total size in bytes:  0


In [28]:
# Extract the tar file
tar_filename = f"data/{filename}.tar"
extract_path = os.path.join('data', filename)

extract_tar(tar_filename, extract_path)

/tmp/ipython-input-1781054501.py:49: DeprecationWarning:

Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.



Successfully extracted data/LitPop_v1_2_orig.tar to data/LitPop_v1_2_orig


'data/LitPop_v1_2_orig'

In [29]:
#"LitPop_pc_30arcsec_GEO.csv"
if DATA_SOURCE == "normalized":
    data_path = "{}/data/LitPop_v1_2_norm".format('/content' if ON_COLAB else '')
    file_path = data_path + "/Lit_norm_30arcsec_FRA.csv"
elif DATA_SOURCE == "original":
    data_path = "{}/data/LitPop_v1_2_orig/LitPop_v1_2".format('/content' if ON_COLAB else '')
    file_path = data_path + "/LitPop_pc_30arcsec_GEO.csv"
else:
  raise ValueError(f"Invalid DATA_SOURCE: {DATA_SOURCE}")

# Load the CSV file
env_df = pd.read_csv(file_path)

# Print the columns
check_cols_in(file_path,
              required_cols = ["location_id", "latitude", "longitude", "population density"])

An error occurred while reading the CSV file: 'str' object has no attribute 'columns'


'/content/data/LitPop_v1_2_orig/LitPop_v1_2/LitPop_pc_30arcsec_GEO.csv'

In [30]:
display(env_df.head())

,value,latitude,longitude,region_id
0,3.170126e+06,43.412500,39.995833,268
1,4.038128e+06,43.404167,39.995833,268
2,4.038684e+06,43.395833,39.995833,268
3,2.026684e+06,43.387500,39.995833,268
4,3.200788e+06,43.429167,40.004167,268


In [31]:

# Initialize SparkSession
spark = SparkSession.builder.appName("AggregateEnvironmentalData").getOrCreate()

# List all CSV files in the extracted directory, excluding hidden files
csv_files = [os.path.join(data_path, f) for f in os.listdir(data_path) if f.endswith('.csv') and not f.startswith('.')]

env_spark_df = csv_to_spark(spark, csv_files)

print("\nSpark DataFrame created successfully.")
env_spark_df.printSchema()
env_spark_df.show(5)


Reading 224 CSV files...


Loading CSV files:   0%|          | 0/224 [00:00<?, ?it/s]


Spark DataFrame created successfully.
root
 |-- value: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- region_id: integer (nullable = true)

+-----+------------------+-------------------+---------+
|value|          latitude|          longitude|region_id|
+-----+------------------+-------------------+---------+
|  0.0|13.337500000000004| -16.82916666666668|      270|
|  0.0|13.345833333333333|-16.820833333333326|      270|
|  0.0|13.337500000000004|-16.820833333333326|      270|
|  0.0|13.329166666666666|-16.820833333333326|      270|
|  0.0| 13.32083333333334|-16.820833333333326|      270|
+-----+------------------+-------------------+---------+
only showing top 5 rows



### Exercise 2: Spatial Join or Nearest Neighbor Matching
**Task:** Merge environmental data with steel plants based on geographic proximity.
- Use nearest neighbor matching or spatial join
- Consider using geopandas for distance calculations
- Match each plant to the nearest environmental monitoring station


In [32]:
# Ensure 'operating_df' has latitude and longitude as numeric
to_num = lambda df, k : pd.to_numeric(df[k], errors='coerce')
operating_df['latitude'], operating_df['longitude'] = to_num(operating_df, "latitude"), to_num(operating_df, "longitude")

# Drop rows with missing coordinates in operating_df and explicitly create a copy
operating_df_cleaned = operating_df.dropna(subset=['latitude', 'longitude']).copy()

# Convert operating_df to a GeoDataFrame
geometry_plants = [Point(xy) for xy in zip(operating_df_cleaned['longitude'], operating_df_cleaned['latitude'])]
plants_gdf = gpd.GeoDataFrame(operating_df_cleaned, geometry=geometry_plants)

# Convert env_df to a GeoDataFrame
# Assuming env_df has 'latitude' and 'longitude' columns and is already loaded
geometry_env = [Point(xy) for xy in zip(env_df['longitude'], env_df['latitude'])]
env_gdf = gpd.GeoDataFrame(env_df, geometry=geometry_env)

# Ensure both GeoDataFrames have the same CRS (Coordinate Reference System)
# A common one for global data is WGS84 (EPSG:4326)
plants_gdf = plants_gdf.set_crs(epsg=4326, allow_override=True)
env_gdf = env_gdf.set_crs(epsg=4326, allow_override=True)


# Perform nearest neighbor join
# Use the env_gdf GeoDataFrame directly in sjoin_nearest
merged_spatial_df = plants_gdf.sjoin_nearest(env_gdf, how="left")

# Display the first few rows of the merged data
print("Merged data with environmental information:")
display(merged_spatial_df.head())

/tmp/ipython-input-3328094457.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Merged data with environmental information:


/usr/local/lib/python3.12/dist-packages/geopandas/array.py:408: UserWarning:

Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.




,Plant name (English)_x,Plant name (other language)_x,Other plant names (English),Other plant names (other language),Owner,Owner (other language),Owner GEM ID,Owner PermID,SOE Status,Parent,...,Nominal DRI capacity (ttpa),Other/unspecified iron capacity (ttpa),latitude_left,longitude_left,geometry,index_right,value,latitude_right,longitude_right,region_id
Plant ID,,,,,,,,,,,,,,,,,,,,,
P100000120439,Algerian Qatari Steel Jijel plant,الجزائرية القطرية للصلب,AQS,NaN,Algerian Qatari Steel,NaN,E100001000957,5076384326,Partial,Industries Qatar QPSC [49.0%]; unknown [5.0%];...,...,2500.0,NaN,36.753961,6.244420,POINT (6.24442 36.75396),3,2.026684e+06,43.3875,39.995833,268
P100000121198,Ozmert Algeria steel plant,NaN,NaN,NaN,Ozmert Algeria SARL,NaN,E100001012196,unknown,NaN,Ozmert Algeria SARL [100.0%],...,500.0,NaN,35.478580,-0.621390,POINT (-0.62139 35.47858),3,2.026684e+06,43.3875,39.995833,268
P100000120440,Sider El Hadjar Annaba steel plant,مركب الحجار للحديد والصلب,"ArcelorMittal Annaba (predecessor), El Hadjar ...",NaN,Groupe Industriel Sider SpA,NaN,E100001000960,5000941519,Full,Groupe Industriel Sider SpA [100.0%],...,NaN,NaN,36.795924,7.707633,POINT (7.70763 36.79592),3,2.026684e+06,43.3875,39.995833,268
P100000120441,Tosyali Algerie Oran steel plant,شركة توسيالي الجزائرية التركية للحديد والصلب,NaN,Tosyali Algérie,Tosyali Ironsteel Industry Algerie SpA,NaN,E100000131071,5074196906,NaN,Tosyalı Holding AŞ,...,5000.0,NaN,35.770912,-0.263031,POINT (-0.26303 35.77091),3,2.026684e+06,43.3875,39.995833,268
P100000120005,Aceria Angola Bengo steel plant,NaN,ADA Steel,NaN,Ada - Aceria De Angola SA,NaN,E100000131097,unknown,NaN,Sa Machado Lda,...,NaN,NaN,-8.582400,13.405271,POINT (13.40527 -8.5824),9251,0.000000e+00,41.5125,41.529167,268


### Exercise 3: Visualize Merged Data
**Task:** Create a map showing steel plants colored by environmental metrics.
- Color plants by air quality index or other environmental indicators
- Size by capacity
- Add hover details with both plant and environmental information


In [33]:
fig = getFig_basic_scatter(
    merged_spatial_df,
    title="Operating Steel Plants by Capacity and Environmental Metric",
    color="value",
    hover_data={"Owner": True,"Nominal crude steel capacity (ttpa)": True,"value": True, "latitude_left": False,"longitude_left": False,"Country/Area_x": True,"region_id": True},
    lat="latitude_left",
    lon="longitude_left")

fig.show()

In [34]:
merged_spatial_df.head()

,Plant name (English)_x,Plant name (other language)_x,Other plant names (English),Other plant names (other language),Owner,Owner (other language),Owner GEM ID,Owner PermID,SOE Status,Parent,...,Nominal DRI capacity (ttpa),Other/unspecified iron capacity (ttpa),latitude_left,longitude_left,geometry,index_right,value,latitude_right,longitude_right,region_id
Plant ID,,,,,,,,,,,,,,,,,,,,,
P100000120439,Algerian Qatari Steel Jijel plant,الجزائرية القطرية للصلب,AQS,NaN,Algerian Qatari Steel,NaN,E100001000957,5076384326,Partial,Industries Qatar QPSC [49.0%]; unknown [5.0%];...,...,2500.0,NaN,36.753961,6.244420,POINT (6.24442 36.75396),3,2.026684e+06,43.3875,39.995833,268
P100000121198,Ozmert Algeria steel plant,NaN,NaN,NaN,Ozmert Algeria SARL,NaN,E100001012196,unknown,NaN,Ozmert Algeria SARL [100.0%],...,500.0,NaN,35.478580,-0.621390,POINT (-0.62139 35.47858),3,2.026684e+06,43.3875,39.995833,268
P100000120440,Sider El Hadjar Annaba steel plant,مركب الحجار للحديد والصلب,"ArcelorMittal Annaba (predecessor), El Hadjar ...",NaN,Groupe Industriel Sider SpA,NaN,E100001000960,5000941519,Full,Groupe Industriel Sider SpA [100.0%],...,NaN,NaN,36.795924,7.707633,POINT (7.70763 36.79592),3,2.026684e+06,43.3875,39.995833,268
P100000120441,Tosyali Algerie Oran steel plant,شركة توسيالي الجزائرية التركية للحديد والصلب,NaN,Tosyali Algérie,Tosyali Ironsteel Industry Algerie SpA,NaN,E100000131071,5074196906,NaN,Tosyalı Holding AŞ,...,5000.0,NaN,35.770912,-0.263031,POINT (-0.26303 35.77091),3,2.026684e+06,43.3875,39.995833,268
P100000120005,Aceria Angola Bengo steel plant,NaN,ADA Steel,NaN,Ada - Aceria De Angola SA,NaN,E100000131097,unknown,NaN,Sa Machado Lda,...,NaN,NaN,-8.582400,13.405271,POINT (13.40527 -8.5824),9251,0.000000e+00,41.5125,41.529167,268


---
## Part 5: Company-Level Aggregation

Aggregate data at the company level to analyze corporate footprints.


### Exercise 1: Aggregate Metrics by Company
**Task:** Group plants by company and calculate aggregate metrics.
- Total capacity per company
- Number of plants per company
- Average environmental metrics per company
- Geographic spread (e.g., number of countries)


#### Hyperparams

In [35]:
NB_TOP_COMPANIES = 20

#### Functions

In [36]:
operating_df.index

Index(['P100000120439', 'P100000121198', 'P100000120440', 'P100000120441',
       'P100000120005', 'P100000121215', 'P100000120006', 'P100000120009',
       'P100000120007', 'P100000120008',
       ...
       'P100001045582', 'P100000121017', 'P100000121016', 'P100000121018',
       'P100001045591', 'P100001045592', 'P100000121023', 'P100001045578',
       'P100000121021', 'P100000121031'],
      dtype='object', name='Plant ID', length=993)

#### Run

In [37]:
# Group by company and aggregate metrics
company_agg = operating_df.reset_index().groupby("Owner").agg(
    total_capacity=("Nominal crude steel capacity (ttpa)", "sum"),
    number_of_plants=("Plant ID", "count"),
    number_of_countries=("Country/Area_x", "nunique")
).reset_index()

# Sort by total capacity
company_agg = company_agg.sort_values("total_capacity", ascending=False)

print("Company-level Aggregate Metrics:")
display(company_agg.head(20)) # Display top 20 companies


Company-level Aggregate Metrics:


,Owner,total_capacity,number_of_plants,number_of_countries
477,Nippon Steel Corp,44423.0,11,1
503,POSCO Holdings Inc,40700.0,2,1
44,Angang Steel Co Ltd,30250.0,3,1
348,JFE Steel Corp,28468.0,5,1
351,JSW Steel Ltd,28359.0,5,1
144,Cleveland-Cliffs Inc,26377.0,11,2
330,Hyundai Steel Co,24297.0,3,1
630,Steel Authority of India Ltd,21050.0,8,1
103,Baoshan Iron & Steel Co Ltd,19800.0,1,1
661,Tata Steel Ltd,19720.0,3,2


In [38]:
# Plot top 20 companies by total capacity
# I decided to keep teh same order and not sort them such that teh user can follow a given company over the plots

top_companies = company_agg.head(NB_TOP_COMPANIES)

getFig_toComp_Owner = lambda y, t : px.bar(top_companies, x="Owner", y=y, title=f"Top {NB_TOP_COMPANIES} Companies by {t}")

fig_capacity = getFig_toComp_Owner("total_capacity", "Total Crude Steel Capacity (ttpa)")
fig_capacity.show()

fig_plants = getFig_toComp_Owner("number_of_plants", "Number of Operating Steel Plants")
fig_plants.show()

# Plot top 20 companies by number of countries
fig_countries = getFig_toComp_Owner("number_of_countries", "Number of Countries")
fig_countries.show()


### Exercise 2: Company Headquarters or Centroid
**Task:** Calculate a representative location for each company.
- Option 1: Use the centroid of all plant locations
- Option 2: Use the location of the largest plant
- Option 3: Assign actual headquarters coordinates


In [39]:
# Calculate representative location for each company (using the location of the largest plant)
# Find the largest plant for each company based on crude steel capacity
largest_plant_per_company = operating_df.loc[operating_df.groupby("Owner")["Nominal crude steel capacity (ttpa)"].idxmax()]

# Select relevant columns for the representative location
company_locations = largest_plant_per_company[["Owner", "latitude", "longitude", "Nominal crude steel capacity (ttpa)", "Plant name (English)_x"]]

print("Representative Location (Largest Plant) for Each Company:")
display(company_locations.head())

Representative Location (Largest Plant) for Each Company:


,Owner,latitude,longitude,Nominal crude steel capacity (ttpa),Plant name (English)_x
Plant ID,,,,,
P100000120979,A. Finkl & Sons Corp,41.724915,-87.589968,573.0,Finkl Steel Chicago plant
P100000120882,ABA Çelik Demir LŞ,36.747413,36.217330,1100.0,Aba Iron and Steel Payas plant
P100000120624,AFV Acciaierie Beltrame SpA,45.522097,11.499295,1200.0,AFV Accaierie Beltrame Vicenza steel plant
P100000120455,AG Siderurgica Balboa SA,38.326405,-6.767326,1300.0,Grupo Gallardo Siderurgica Balboa Jerez de los...
P100000120423,AG der Dillinger Hüttenwerke AG,49.353884,6.746603,2760.0,AG der Dillinger Hüttenwerke Dillingen steel p...


### Exercise 3: Visualize Company-Level Data
**Task:** Create a map showing companies with aggregated metrics.
- Show one marker per company at the representative location
- Size by total capacity
- Color by average environmental impact
- Hover information with company summary statistics


In [40]:
company_map_data = pd.merge(company_locations, company_agg, on="Owner")

fig = getFig_basic_scatter(
    company_map_data,
    title="Operating Steel Companies by Total Capacity and Number of Plants",
    color="number_of_plants",
    hover_data={"total_capacity": True, "number_of_plants": True, "number_of_countries": True, "Plant name (English)_x": True, "latitude": False,"longitude": False},
    size="total_capacity")

fig.show()

---
## Part 6: Streamlit Dashboard Integration

Prepare your visualizations for deployment in a Streamlit dashboard.


### Exercise 1: Create Dashboard Script Structure
**Task:** Create a Streamlit app file (`dashboard.py`) with the following structure:

```python
# Import streamlit and other necessary libraries

# Set page configuration

# Title and description

# Sidebar for filters
# - Company selector
# - Region/country filter
# - Capacity range slider

# Main content area
# - KPI metrics (total plants, total capacity, etc.)
# - Interactive map
# - Data table

# Footer with data sources and notes
```


### Exercise 1: Prepare Data for Dashboard
**Task:** Save your processed data to files that the dashboard will load.
- Export cleaned plant data
- Export merged environmental data
- Export company-level aggregations
- Save as CSV or Parquet for efficient loading


#### Hyperparams

In [41]:
# Define paths for saving the dataframes
operating_plants_path = "data/operating_plants.csv"
merged_environmental_path = "data/merged_environmental_data.csv"
company_aggregation_path = "data/company_aggregation.csv"

#### Run

In [42]:

# Save the dataframes to CSV
operating_df_cleaned.to_csv(operating_plants_path, index=False)
print(f"Cleaned operating plants data saved to {operating_plants_path}")


merged_spatial_df.drop(columns=['geometry']).to_csv(merged_environmental_path, index=False)
print(f"Merged environmental data saved to {merged_environmental_path}")

company_agg.to_csv(company_aggregation_path, index=False)
print(f"Company aggregation data saved to {company_aggregation_path}")

Cleaned operating plants data saved to data/operating_plants.csv
Merged environmental data saved to data/merged_environmental_data.csv
Company aggregation data saved to data/company_aggregation.csv


### Exercise 2: Display relevant information from your exploratory analysis into the dashboard

In [ ]:
# This cell is for notes/observations about your dashboard
# What works well?
# What could be improved?
# Any performance issues with large datasets?

"""
1) I think the dashboard is quite good and complete, it has all the necessary information and is quite easy to use. I would love to add a teh possibility to interact with a chatboot so the user can build its own visuals and ask any question regarding the dataset. 
2) the chatbot implementation could be improve, also enabling the user to add new fiel and do cross analysis. 
3) The main issue was downloading it using in a comprenhensive and efficient way. For loading and treatment i used SPARK as it is a very capable tool for big data handling. 
"""

---
## Lab Summary and Key Takeaways

**What you learned:**
- How to perform EDA on geospatial datasets
- Creating interactive maps with Plotly for geospatial data
- Merging spatial datasets based on geographic proximity
- Aggregating geospatial data at different levels (asset vs. company)
- Building interactive dashboards with Streamlit

**Next Steps:**
- Explore other geospatial libraries (GeoPandas, Folium, Kepler.gl)
- Learn about coordinate reference systems (CRS) and projections
- Practice with other datasets (buildings, utilities, transportation)
- Deploy your dashboard to Streamlit Cloud or other hosting services
